In [ ]:
from selenium import webdriver
from fake_useragent import UserAgent
from proxy import Proxy
import os

def make_driver():

    proxy = Proxy()
    print('adding a proxy')
    while proxy.test_proxy(proxy.proxy) != 1:
        proxy.cycle()
        print(f"Trying new proxy at: {proxy.proxy['http']}")
    prox = proxy.proxy['http'].split('//')[1]
    options = webdriver.ChromeOptions() 
    userAgent = UserAgent().random
    options.add_argument("start-maximized")
    options.add_argument('--proxy-server=%s' % prox)
    options.add_argument(f'user-agent={userAgent}')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=options, executable_path= os.path.split(__file__)[0] + "\\chromedriver.exe")
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
        """
    })
    return driver

In [1]:
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [2]:
Jobsearch = "Data scientist"
Where = "London "
daysfilter = "24hours"

In [ ]:
driver = make_driver()
driver.get("http://whatismyipaddress.com")
driver.get("https://uk.indeed.com/")
driver.find_element_by_xpath('//*[@id="text-input-what"]').send_keys(Jobsearch)
driver.find_element_by_xpath('//*[@id="text-input-where"]').send_keys(Where)
driver.find_element_by_xpath('//*[@id="whatWhereFormId"]/div[3]/button').click()
driver.find_element_by_xpath('//*[@id="filter-dateposted"]/button').click()
if len(daysfilter) > 1:
    if daysfilter == '24hours':
        driver.find_element_by_xpath('//*[@id="filter-dateposted-menu"]/li[1]/a').click()
    elif daysfilter == '3days':
        driver.find_element_by_xpath('//*[@id="filter-dateposted-menu"]/li[2]/a').click()
    elif daysfilter == '7days':
        driver.find_element_by_xpath('//*[@id="filter-dateposted-menu"]/li[3]/a').click()
    elif daysfiltr == '14days':
        driver.find_element_by_xpath('//*[@id="filter-dateposted-menu"]/li[4]/a').click()
    try:
        time.sleep(1)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-pc-btn-handler"]'))).click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="cookie-preferences"]/div[5]/button'))).click()
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'popover-x')))
        element.click()
    except:
        print('could not find pop over')

def scrape_indeed():
    el = driver.find_elements_by_class_name('title')
    predf = {'Title': [], 'Company': [], 'Location' : [], 'Money' : [], 'Description' : []}
    for i in el:
        i.click()
        predf['Title'].append(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'vjs-jobtitle'))).text)
        predf['Company'].append(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="vjs-cn"]'))).text)
        predf['Location'].append(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="vjs-loc"]'))).text)
        try:
            predf['Money'].append(driver.find_element_by_xpath('//*[@id="vjs-jobinfo"]/div[3]').text)
        except:
            predf['Money'].append(0)
        predf['Description'].append(driver.find_element_by_xpath('//*[@id="vjs-desc"]').text)

        return pd.DataFrame(predf)

    
dataframes = []    
while driver.find_element_by_xpath('//*[@id="resultsCol"]/nav/div/ul/li[4]/a').size() != 0:
    dataframes.append(scrape_indeed())
    driver.find_element_by_xpath('//*[@id="resultsCol"]/nav/div/ul/li[4]/a').click()
    
df = pd.concat(dataframes)

adding a proxy
Trying new proxy at: http://165.22.81.30:37344
Trying new proxy at: http://85.173.165.36:46330
Trying new proxy at: http://202.151.202.68:3128


,0
0,Easily apply to this job
